# Εργασία 2 (Τεχνικές Εξόρυξης Δεδομένων)
## Data Mining: Assignment 2
***
### Μαρία Φριτζελά 1115201400218
***

In [7]:
import pandas as pd
from unicodedata import normalize
import re

## Collection and cleaning of data
Date information is not needed so it is not added to our dataframes

In [2]:
traindf = pd.read_csv("data/train.csv", usecols=['Insult', 'Comment'])
testdf = pd.read_csv("data/impermium_verification_labels.csv", index_col='id', usecols=['id', 'Insult', 'Comment'])

Looking at traindf:

In [3]:
traindf

,Insult,Comment
0,1,"""You fuck your dad."""
1,0,"""i really don't understand your point.\xa0 It ..."
2,0,"""A\\xc2\\xa0majority of Canadians can and has ..."
3,0,"""listen if you dont wanna get married to a man..."
4,0,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd..."
...,...,...
3942,1,"""you are both morons and that is never happening"""
3943,0,"""Many toolbars include spell check, like Yahoo..."
3944,0,"""@LambeauOrWrigley\xa0\xa0@K.Moss\xa0\nSioux F..."
3945,0,"""How about Felix? He is sure turning into one ..."


Splitting the train and test data to X and y

In [11]:
X_train, y_train = traindf.Comment, traindf.Insult

In [5]:
X_test, y_test = testdf.Comment, testdf.Insult

Clean up train comments' text:
- convert all letters to lowercase
- remove multiple instances of '\'
For example "\\\n" becomes "\n"
- remove "\n"
- remove usernames
- remove URLs
- normalize unicode data like "\\xa0" and others <br>
NFKD will apply the compatibility decomposition, i.e. replace all compatibility characters with their equivalents.
- remove puctuation
- remove all words containing digits, and any digits
- remove multiple spaces


In [126]:
X_train = traindf.Comment.apply(lambda comment: comment.lower())\
                .apply(lambda comment: re.sub("\\\\*\\\\", "\\\\" ,comment))\
                .apply(lambda comment: re.sub("\\\\n", " ", comment))\
                .apply(lambda comment: re.sub('@[^\s]+','',comment))\
                .apply(lambda comment: re.sub('(http(s)?://|www.)(.|[a-z]|[A-Z]| [0-9])+', " ", comment))\
                .apply(lambda comment: normalize("NFKD", comment))\
                .apply(lambda comment: re.sub('[^A-Za-z0-9 ]+', ' ',comment))\
                .apply(lambda comment: re.sub(r'\w*\d\w*', '', comment))\
                .apply(lambda comment: re.sub(r"\s+"," ", comment, flags = re.I))

In [127]:
traindf.Comment[81]

'"M\\xe1tenlos!!\\nhttp://1.bp.blogspot.com/-YVSZmVv2iRw/Tmo8KFaUGMI/AAAAAAAAA0k/NlLAahKNnis/s1600/kill_it_with_fire_KILL_IT_KILL_IT_WITH_FIRE-s670x394-132457.jpg"'

In [128]:
X_train[81]

' m '

In [129]:
traindf.Comment[124]

'"Nope. Not working for me either.32-23-34www.facebook.com/annagillmodel\\\\n\\\\n \\\\n\\\\nYou have my email! :) "'

In [130]:
X_train[124]

' nope not working for me either '

In [132]:
X_train[91]

' they should put all the gay and lesbians on an island oh wait there is manhattan '

In [131]:
X_train

0                                      you fuck your dad 
1        i really don t understand your point it seems...
2        a of canadians can and has been wrong before ...
3        listen if you dont wanna get married to a man...
4        c b xu bi t xecnh c ho kh c ng d ng cu xed ch...
                              ...                        
3942     you are both morons and that is never happening 
3943     many toolbars include spell check like yahoo ...
3944     sioux falls s d i told my boy he should call ...
3945     how about felix he is sure turning into one h...
3946     you re all upset defending this hipster band ...
Name: Comment, Length: 3947, dtype: object

In [34]:
normalize("NFKD", "@LambeauOrWrigley\xa0\xa0@K.Moss\xa0")

'@LambeauOrWrigley  @K.Moss '

In [45]:
normalize("NFKD", "on an island\xe2\x80\xa6\xc2\xa0 Oh wait there is Manhattan.")

'on an islandâ\x80¦Â  Oh wait there is Manhattan.'